Model Deployment: Predict 7 days of close stock price for ticker AQN.TO using TF Serving\
Data source: Yahoo Finance\
Created by: Gary Sampson DatumSam AI

In [1]:
import yfinance as yf
import pandas as pd

In [2]:
# Download Algonquin Power & Utilities Corp. and S&P/TSX composite index from January 1,2010 to October 28, 2022
df = yf.download(['AQN.TO', '^GSPTSE'], start='2010-01-01', end='2022-10-28', 
                    progress=False, auto_adjust=True)
df1 = df['Close']
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3217 entries, 2010-01-04 to 2022-10-27
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   AQN.TO   3217 non-null   float64
 1   ^GSPTSE  3217 non-null   float64
dtypes: float64(2)
memory usage: 75.4 KB


In [3]:
# make a copy of dataset df1
df2 = df1.copy()

In [4]:
# Original dataset did not have weekend and holiday data. Impute with forward fill data to produce timeseries dataset 
Date_new = pd.date_range(start='2010-01-04', end='2022-10-28', freq='D')
df2 = df2.reindex(Date_new, method='ffill' )

In [5]:
# Split the dataset into train, validate and test sets
df_train = df2["2010-01-04":"2019-12-31"]
df_valid = df2["2021-01-01":"2022-09-01"]
df_test = df2["2022-09-01":]

In [6]:
# Get the mean and standard deviation for the train set 
train_mean = df_train.mean()
train_std = df_train.std()

In [7]:
# Get one test instance
X_new =  df_test[:30].to_numpy().reshape(1, df_test[:30].shape[0], df_test[:30].shape[1])

#### Querying TF Serving through the REST API

In [60]:
import json

# Create the query
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [61]:
request_json

'{"signature_name": "serving_default", "instances": [[[17.29204559326172, 19142.69921875], [17.23459815979004, 19270.900390625], [17.23459815979004, 19270.900390625], [17.23459815979004, 19270.900390625], [17.23459815979004, 19270.900390625], [17.11969757080078, 19088.19921875], [17.263320922851562, 19241.400390625], [17.244173049926758, 19413.0], [17.14842414855957, 19773.30078125], [17.14842414855957, 19773.30078125], [17.14842414855957, 19773.30078125], [17.320770263671875, 19987.19921875], [17.033527374267578, 19645.400390625], [17.01437759399414, 19726.099609375], [16.985652923583984, 19560.19921875], [16.784584045410156, 19385.900390625], [16.784584045410156, 19385.900390625], [16.784584045410156, 19385.900390625], [16.775009155273438, 19562.400390625], [16.382442474365234, 19368.69921875], [16.411165237426758, 19184.5], [16.382442474365234, 19002.69921875], [16.248394012451172, 18481.0], [16.248394012451172, 18481.0], [16.248394012451172, 18481.0], [15.683483123779297, 18327.0],

In [73]:
import requests

# TF Serving's API to make predictions
server_url = "http://localhost:8501/v1/models/best_multivariate_rnn:predict"
response_rest = requests.post(server_url, data=request_json)
response_rest.raise_for_status()  # raise an exception in case of error
response_rest = response_rest.json()

In [74]:
import numpy as np

y_predict_rest = np.array(response_rest["predictions"])[0, -1][-5:]
y_predict_rest.round(2)

array([2.33, 2.35, 2.37, 2.46, 2.43])

In [76]:
# de-nornalise the prediction from Oct 3 - 7, 2022
print('AQN predict from Oct 3 -7,2022:',((y_predict_rest*train_std[0]) + train_mean[0]))

AQN predict from Oct 3 -7,2022: [15.08342985 15.16264613 15.2364106  15.53123115 15.43134676]


In [14]:
print('Actual AQN close stock price from Oct 3 -7, 2022', df_test["2022-10-03":"2022-10-07"]['AQN.TO'])

Actual AQN close stock price from Oct 3 -7, 2022 2022-10-03    15.161493
2022-10-04    15.424071
2022-10-05    15.025341
2022-10-06    14.480734
2022-10-07    14.334856
Freq: D, Name: AQN.TO, dtype: float64


#### Querying TF Serving through the gRPC API

In [77]:
import tensorflow as tf
best_model = tf.keras.models.load_model('c:/users/garys/Documents/Time_Series_Forecasting/best_multivariate_rnn/0003')

In [79]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest
model_name = "best_multivariate_rnn"

request_grpc = PredictRequest()
request_grpc.model_spec.name = model_name
request_grpc.model_spec.signature_name = "serving_default"
input_name = best_model.input_names[0]  # best_model from SavedModel format
request_grpc.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new.tolist()))

In [80]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2.PredictionServiceStub(channel)
response_grpc = predict_service.Predict(request, timeout=10.0)

In [81]:
output_name = best_model.output_names[0]
outputs_proto = response_grpc.outputs[output_name]
y_predict_grpc = tf.make_ndarray(outputs_proto)[0, -1][-5:]
y_predict_grpc

array([2.3310993, 2.3535478, 2.3744514, 2.4579985, 2.429693 ],
      dtype=float32)

In [84]:
# de-nornalise the prediction from Oct 3 - 7, 2022
print('AQN predict from Oct 3 - 7,2022:',((y_predict_grpc*train_std[0]) + train_mean[0]))

AQN predict from Oct 3 - 7,2022: [15.083429 15.162645 15.23641  15.531231 15.431347]


Prediction results are the same for querying through the REST API and gRPC API but it is recommended to use gRPC API when transferring large amounts of data.